In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tabulate import tabulate
import mapply
mapply.init(n_workers=7, progressbar=True)


# Read the data
INCIDENTS = '../../dataset/data-raw/incidents.csv'
POVERTYYEAY = '../../dataset/data-raw/povertyByStateYear.csv'
STATEDISHOUSE = '../../dataset/data-raw/year_state_district_house.csv'

incidents = pd.read_csv(INCIDENTS)
poverty = pd.read_csv(POVERTYYEAY)
state_district_house = pd.read_csv(STATEDISHOUSE)

incidents['state'] = incidents['state'].str.upper()
poverty['state'] = poverty['state'].str.upper()
state_district_house['state'] = state_district_house['state'].str.upper()

incidents['date_year'] = incidents['date'].str.split('-').str[0].astype(int)
incidents['date_month'] = incidents['date'].str.split('-').str[1].astype(int)
incidents['date_day'] = incidents['date'].str.split('-').str[2].astype(int)
del incidents['date']

incidents = incidents[['date_year', 'date_month', 'date_day', 'state', 'city_or_county', 'latitude', 'longitude', 'congressional_district', 'min_age_participants', 'avg_age_participants', 'max_age_participants', 'n_participants_child', 'n_participants_teen', 'n_participants_adult', 'n_males', 'n_females', 'n_killed', 'n_injured', 'n_arrested', 'n_unharmed', 'n_participants', 'incident_characteristics1']].copy()


/tmp/ipykernel_22162/3812903467.py:14: DtypeWarning: Columns (15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  incidents = pd.read_csv(INCIDENTS)


In [4]:
# set non-numeric values to NaN in columns supposed to be numeric

objcols = incidents.select_dtypes(include=['object']).columns
objcols = objcols.drop(['state', 'city_or_county', 'incident_characteristics1'])

for col in objcols:
    incidents[col] = incidents[col].mapply(pd.to_numeric, errors='coerce')

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


...

  0%|          | 0/56 [00:00<?, ?it/s]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

In [67]:
# Infer the congressional district when missing as the most common one in the city
# TO-OPTIMIZE: it would be faster to crack RSA-2048 on a TI-84
def find_cd(row):
    df = incidents[(incidents['state'] == row['state']) * (incidents['city_or_county'] == row['city_or_county'])]
    # find the most common congressional district
    vc = df['congressional_district'].value_counts()
    if vc.empty:
        row['congressional_district'] = np.nan
        return row
    cd = vc.idxmax()
    row['congressional_district'] = cd
    return row

# set out-of-semnsible ranga values to NaN
def sensibleValues(row):
    # --- range checks ---
    if row['date_year'] > 2020:
        row['date_year'] = np.nan
    if row['min_age_participants'] > 110 or row['min_age_participants'] < 0:
        row['min_age_participants'] = np.nan
    if row['avg_age_participants'] > 110 or row['avg_age_participants'] < 0:
        row['avg_age_participants'] = np.nan
    if row['max_age_participants'] > 110 or row['max_age_participants'] < 0:
        row['max_age_participants'] = np.nan
    # 103 is the maximum value of n_participants
    if row['n_participants_child'] > 103 or row['n_participants_child'] < 0:
        row['n_participants_child'] = np.nan
    if row['n_participants_teen'] > 103 or row['n_participants_teen'] < 0:
        row['n_participants_teen'] = np.nan
    if row['n_participants_adult'] > 103 or row['n_participants_adult'] < 0:
        row['n_participants_adult'] = np.nan
    if row['n_killed'] > 103 or row['n_killed'] < 0:
        row['n_killed'] = np.nan
    if row['n_injured'] > 103 or row['n_injured'] < 0:
        row['n_injured'] = np.nan
    if row['n_arrested'] > 103 or row['n_arrested'] < 0:
        row['n_arrested'] = np.nan
    if row['n_unharmed'] > 103 or row['n_unharmed'] < 0:
        row['n_unharmed'] = np.nan
    return row


sincidents = incidents.mapply(sensibleValues, axis=1)

sincidents['state_city'] = sincidents['state'] + "-" + sincidents['city_or_county']
cd_df = sincidents[['state_city', 'congressional_district']]
cd_vc = cd_df.groupby('state_city')['congressional_district'].value_counts()
cd_vc = cd_vc.reset_index(name='count')
cd_vc = cd_vc.groupby('state_city').head(1)

def fix_cd(row):
    if np.isnan(row['congressional_district']):
        should_be = cd_vc[cd_vc['state_city'] == row['state_city']]
        if should_be.empty:
            row['congressional_district'] = np.nan
        else:
            row['congressional_district'] = should_be['congressional_district'].values[0]
    return row

sincidents = sincidents.mapply(fix_cd, axis=1)
sincidents = sincidents.drop('state_city', axis=1)

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

In [68]:
# given the groups 
#   (n_participants)
#   (n_males, n_females)
#   (n_participants_child, n_participants_teen, n_participants_adult)
#   (n_killed, n_injured, n_arrested, n_unharmed)
# we find and solve inconsistencies between their sums, keeping the original proportions
# Example:
#   n_participants = 10
#   n_participants_child
#       + n_participants_teen
#       + n_participants_adult = 10
#   n_males = 2
#   n_females = 6
# then the most probable values (according to our assumptions) are groups that sum to 10 so:
#   n_males => 3
#   n_females => 7

from random import randint

# Don't ask, it works (maybe)
def mf(row):
    mf = row['n_males'] + row['n_females']
    cta = row['n_participants_child'] + row['n_participants_teen'] + row['n_participants_adult']
    kiau = row['n_killed'] + row['n_injured'] + row['n_arrested'] + row['n_unharmed']
    p = row['n_participants']

    if mf == cta == kiau == p:
        return 0b0000
    elif mf == cta == kiau:
        return 0b0001
    elif mf == cta == p:
        return 0b0010
    elif mf == kiau == p:
        return 0b0100
    elif cta == kiau == p:
        return 0b1000
    elif mf == p:
        return 0b0110
    elif cta == p:
        return 0b1010
    elif kiau == p:
        return 0b1100
    elif mf == cta:
        return 0b0011
    elif mf == kiau:
        return 0b0101
    elif cta == kiau:
        return 0b1001
    return 0b1111

def perc_calc(vals: list):
    s = np.sum(vals)
    if s == 0:
        return 1/len(vals)
    return vals[0] / s

    
def nannifyErrors(row):
    err_bits = mf(row)
    err_p = err_bits & 0b0001
    err_mf = err_bits & 0b1000
    err_cta = err_bits & 0b0100
    err_kiau = err_bits & 0b0010

    ref = row['n_participants'] if not err_p else \
        row['n_males'] + row['n_females'] if not err_mf else \
        row['n_participants_child'] + row['n_participants_teen'] + row['n_participants_adult'] if not err_cta else \
        row['n_killed'] + row['n_injured'] + row['n_arrested'] + row['n_unharmed'] if not err_kiau else \
        np.nan


    if err_p:
        row['n_participants'] = ref
    if err_mf:
        p_m = perc_calc([row['n_males'], row['n_females']])
        row['n_males'] = np.round(ref * p_m)
        row['n_females'] = np.round(ref * (1 - p_m))
        diff = ref - (row['n_males'] + row['n_females'])
        if diff != 0:
            # With 1/2 probability add the difference to males of females
            row['n_males' if randint(0,1) else 'n_females'] += diff
    if err_cta:
        p_c = perc_calc([row['n_participants_child'], row['n_participants_teen'], row['n_participants_adult']])
        p_t = perc_calc([row['n_participants_teen'], row['n_participants_child'], row['n_participants_adult']])
        row['n_participants_child'] = np.round(ref * p_c)
        row['n_participants_teen'] = np.round(ref * p_t)
        row['n_participants_adult'] = np.round(ref * (1 - p_c - p_t))
        diff = ref - (row['n_participants_child'] + row['n_participants_teen'] + row['n_participants_adult'])
        if diff != 0:
            # With 1/3 probability, add the difference to child, teen or adult
            row['n_participants_child' if randint(0,2) == 0 else 'n_participants_teen' if randint(0,1) else 'n_participants_adult'] += diff
    if err_kiau:
        p_k = perc_calc([row['n_killed'], row['n_injured'], row['n_arrested'], row['n_unharmed']])
        p_i = perc_calc([row['n_injured'], row['n_killed'], row['n_arrested'], row['n_unharmed']])
        p_a = perc_calc([row['n_arrested'], row['n_killed'], row['n_injured'], row['n_unharmed']])
        row['n_killed'] = np.round(ref * p_k)
        row['n_injured'] = np.round(ref * p_i)
        row['n_arrested'] = np.round(ref * p_a)
        row['n_unharmed'] = np.round(ref * (1 - p_k - p_i - p_a))
        diff = ref - (row['n_killed'] + row['n_injured'] + row['n_arrested'] + row['n_unharmed'])
        if diff != 0:
            # With 1/4 probability, add the difference to killed, injured, arrested or unharmed
            row['n_killed' if randint(0,3) == 0 else 'n_injured' if randint(0,2) == 0 else 'n_arrested' if randint(0,1) else 'n_unharmed'] += diff
    return row



diff = sincidents.mapply(mf, axis=1)
print('DONE diff')
# incidents without problems
dncidents = sincidents[diff == 0b0000]
print('DONE dncidents')
# incidents with problems
wncidents = sincidents[diff != 0b0000].mapply(nannifyErrors, axis=1)
print('DONE wncidents')

# dncidents = incidents.iloc[0:10000].apply(nannifyErrors, axis=1)

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

DONE diff
DONE dncidents


/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

DONE wncidents


In [69]:
# merge wncidents with dncidents
wdincidents = pd.concat([wncidents, dncidents], ignore_index=False)
# shuffle the rows to mix the incidents with problems (wncidents) with the others (dncidents)
wdincidents = wdincidents.sample(frac=1)

In [70]:
cincidents = wdincidents.copy()

print(tabulate(pd.DataFrame(cincidents.isnull().sum(axis=0)).T, headers='keys', tablefmt='psql'))

# cincidents['n_nan'] = cincidents.isnull().sum(axis=1)
# cincidents['n_nan'].value_counts().sort_index().plot.bar()

mfdslkmfdslk = cincidents[(cincidents[['n_participants', 'n_males', 'n_females', 'n_participants_child', 'n_participants_teen', 'n_participants_adult', 'n_killed', 'n_injured', 'n_arrested', 'n_unharmed']].isnull().sum(axis=1) == 10)]

# 425 bad rows
cincidents = cincidents[~cincidents['congressional_district'].isnull()]
# 326 bad rows
cincidents = cincidents[~cincidents['incident_characteristics1'].isnull()]
# remove all rows where the n_participants groups are all NaN
cincidents = cincidents[~(cincidents[['n_participants', 'n_males', 'n_females', 'n_participants_child', 'n_participants_teen', 'n_participants_adult', 'n_killed', 'n_injured', 'n_arrested', 'n_unharmed']].isnull().sum(axis=1) == 10)]
print(tabulate(pd.DataFrame(cincidents.isnull().sum(axis=0)).T, headers='keys', tablefmt='psql'))
# drop duplicates
cincidents = cincidents.drop_duplicates()

print(cincidents.shape[0] / incidents.shape[0])

# TEMPORARY: Fix all NaNs with the median of the column
# for col in ['n_participants', 'n_males', 'n_females', 'n_participants_child', 'n_participants_teen', 'n_participants_adult', 'n_killed', 'n_injured', 'n_arrested', 'n_unharmed']:
for col in cincidents.select_dtypes(include=['float64']).columns:
    median = cincidents[col].median()
    cincidents[col] = cincidents[col].fillna(median)


# set as number of days since 2010-01-01
cincidents['timestamp'] = (pd.to_datetime(cincidents[['date_year', 'date_month', 'date_day']].astype(int).astype(str).apply(lambda x: '-'.join(x), axis=1)).astype(int) - pd.to_datetime(['2010-01-01']).astype(int)[0]) / 86400000000000

# cincidents = cincidents.drop(['date_year', 'date_month', 'date_day'], axis=1)
print(tabulate(pd.DataFrame(cincidents.isnull().sum(axis=0)).T, headers='keys', tablefmt='psql'))

+----+-------------+--------------+------------+---------+------------------+------------+-------------+--------------------------+------------------------+------------------------+------------------------+------------------------+-----------------------+------------------------+-----------+-------------+------------+-------------+--------------+--------------+------------------+-----------------------------+
|    |   date_year |   date_month |   date_day |   state |   city_or_county |   latitude |   longitude |   congressional_district |   min_age_participants |   avg_age_participants |   max_age_participants |   n_participants_child |   n_participants_teen |   n_participants_adult |   n_males |   n_females |   n_killed |   n_injured |   n_arrested |   n_unharmed |   n_participants |   incident_characteristics1 |
|----+-------------+--------------+------------+---------+------------------+------------+-------------+--------------------------+------------------------+------------------

In [72]:
# Merge with poverty
# poverty header: [state,year,povertyPercentage]
merged = cincidents.merge(poverty, how='left', left_on=['state', 'date_year'], right_on=['state', 'year'])
merged = merged.drop(columns=['year'])

# Merge with state_district_house
# state_district_house header: [year,state,congressional_district,party,candidatevotes,totalvotes]
# state_district_house rows are only for even years but we have incidents for all years
merged['congressional_district'] = merged['congressional_district'].mapply(lambda x: 1 if x == 0 else x)
state_district_house['congressional_district'] = state_district_house['congressional_district'].mapply(lambda x: 1 if x == 0 else x)
stete_district_house_odd = state_district_house.copy()
stete_district_house_odd['year'] = state_district_house['year'] + 1
state_district_house = pd.concat([state_district_house, stete_district_house_odd], ignore_index=True).reset_index(drop=True)

merged = merged.merge(state_district_house, how='left', left_on=['state', 'date_year', 'congressional_district'], right_on=['state', 'year', 'congressional_district'])
merged = merged.drop(columns=['year'])

merged.isnull().sum(axis=0)
merged = merged[~(merged['congressional_district'] == 0)]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

/home/simone/Unipi/InProgress/DM/DM/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


  0%|          | 0/56 [00:00<?, ?it/s]

In [73]:
merged

,date_year,date_month,date_day,state,city_or_county,latitude,longitude,congressional_district,min_age_participants,avg_age_participants,...,n_injured,n_arrested,n_unharmed,n_participants,incident_characteristics1,timestamp,povertyPercentage,party,candidatevotes,totalvotes
0,2016.0,9,24,CALIFORNIA,Truckee,38.4409,-86.2730,4.0,25.0,27.0,...,0.0,0.0,3.0,4.0,"Shot - Dead (murder, accidental, suicide)",2458.0,12.5,REPUBLICAN,220133.0,350978.0
1,2016.0,9,24,CALIFORNIA,Truckee,38.4409,-86.2730,4.0,25.0,27.0,...,0.0,0.0,3.0,4.0,"Shot - Dead (murder, accidental, suicide)",2458.0,12.5,REPUBLICAN,126784.0,211134.0
2,2016.0,8,31,SOUTH CAROLINA,Rock Hill,34.9277,-80.9862,5.0,25.0,27.0,...,0.0,0.0,2.0,2.0,Non-Shooting Incident,2434.0,14.1,REPUBLICAN,161669.0,273006.0
3,2016.0,8,31,SOUTH CAROLINA,Rock Hill,34.9277,-80.9862,5.0,25.0,27.0,...,0.0,0.0,2.0,2.0,Non-Shooting Incident,2434.0,14.1,REPUBLICAN,103078.0,175145.0
4,2017.0,4,6,INDIANA,Fort Wayne,41.0902,-85.1509,3.0,18.0,18.0,...,2.0,0.0,0.0,2.0,Shot - Wounded/Injured,2652.0,11.1,REPUBLICAN,201396.0,287247.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
422541,2016.0,12,9,TEXAS,Laredo,27.4700,-99.4622,28.0,34.0,34.0,...,0.0,1.0,0.0,1.0,Non-Shooting Incident,2534.0,13.7,DEMOCRAT,62508.0,76136.0
422542,2017.0,7,30,MASSACHUSETTS,Fitchburg,42.5785,-71.7867,3.0,39.0,39.0,...,0.0,1.0,0.0,1.0,Non-Shooting Incident,2767.0,9.2,DEMOCRAT,236713.0,360124.0
422543,2017.0,7,30,MASSACHUSETTS,Fitchburg,42.5785,-71.7867,3.0,39.0,39.0,...,0.0,1.0,0.0,1.0,Non-Shooting Incident,2767.0,9.2,DEMOCRAT,236713.0,360124.0
422544,2018.0,2,21,ALABAMA,Montgomery,32.3559,-86.3270,2.0,25.0,27.0,...,2.0,0.0,0.0,3.0,Shot - Wounded/Injured,2973.0,14.6,REPUBLICAN,138879.0,226230.0
